The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install -U mxnet-cu101==1.7.0


# Camadas Personalizadas

Um fator por trás do sucesso do *Deep Learning*
é a disponibilidade de uma ampla gama de camadas
que pode ser composto de maneiras criativas
projetar arquiteturas adequadas
para uma ampla variedade de tarefas.
Por exemplo, os pesquisadores inventaram camadas
especificamente para lidar com imagens, texto,
loop sobre dados sequenciais,
e
realizando programação dinâmica.
Mais cedo ou mais tarde, você encontrará ou inventará
uma camada que ainda não existe na estrutura de *Deep Learning*.
Nesses casos, você deve construir uma camada personalizada.
Nesta seção, mostramos como.

## Camadas Sem Parâmetros

Para começar, construímos uma camada personalizada
que não possui parâmetros próprios.
Isso deve parecer familiar, se você se lembra de nosso
introdução ao bloco em :numref:`sec_model_construction`.
A seguinte classe `CenteredLayer` simplesmente
subtrai a média de sua entrada.
Para construí-lo, simplesmente precisamos herdar
da classe da camada base e implementar a função de propagação direta.


In [1]:
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def forward(self, X):
        return X - X.mean()

Vamos verificar se nossa camada funciona conforme o esperado, alimentando alguns dados por meio dela.


In [2]:
layer = CenteredLayer()
layer(np.array([1, 2, 3, 4, 5]))

array([-2., -1.,  0.,  1.,  2.])

Agora podemos incorporar nossa camada como um componente
na construção de modelos mais complexos.


In [3]:
net = nn.Sequential()
net.add(nn.Dense(128), CenteredLayer())
net.initialize()

Como uma verificação extra de sanidade, podemos enviar dados aleatórios
através da rede e verificar se a média é de fato 0.
Porque estamos lidando com números de ponto flutuante,
ainda podemos ver um número muito pequeno diferente de zero
devido à quantização.


In [4]:
Y = net(np.random.uniform(size=(4, 8)))
Y.mean()

array(3.783498e-10)

## Camadas com Parâmetros

Agora que sabemos como definir camadas simples,
vamos prosseguir para a definição de camadas com parâmetros
que pode ser ajustado por meio de treinamento.
Podemos usar funções integradas para criar parâmetros, que
fornecem algumas funcionalidades básicas de manutenção.
Em particular, eles governam o acesso, inicialização,
compartilhar, salvar e carregar parâmetros do modelo.
Dessa forma, entre outros benefícios, não precisaremos escrever
rotinas de serialização personalizadas para cada camada personalizada.

Agora, vamos implementar nossa própria versão da camada totalmente conectada.
Lembre-se de que esta camada requer dois parâmetros,
um para representar o peso e outro para o viés.
Nesta implementação, preparamos a ativação do ReLU como padrão.
Esta camada requer a entrada de argumentos: `in_units` e `units`, que
denotam o número de entradas e saídas, respectivamente.


In [5]:
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super().__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = np.dot(x, self.weight.data(ctx=x.ctx)) + self.bias.data(
            ctx=x.ctx)
        return npx.relu(linear)

Em seguida, instanciamos a classe `MyDense`
e acessar seus parâmetros de modelo.


In [6]:
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

Podemos realizar cálculos de propagação direta usando camadas personalizadas.


In [7]:
dense.initialize()
dense(np.random.uniform(size=(2, 5)))

array([[0.        , 0.01633355, 0.        ],
       [0.        , 0.01581812, 0.        ]])

Também podemos construir modelos usando camadas personalizadas.
Assim que tivermos isso, podemos usá-lo como a camada totalmente conectada integrada.


In [8]:
net = nn.Sequential()
net.add(MyDense(8, in_units=64),
        MyDense(1, in_units=8))
net.initialize()
net(np.random.uniform(size=(2, 64)))

array([[0.06508517],
       [0.0615553 ]])

## Sumário

* Podemos projetar camadas personalizadas por meio da classe de camada básica. Isso nos permite definir novas camadas flexíveis que se comportam de maneira diferente de quaisquer camadas existentes na biblioteca.
* Uma vez definidas, as camadas personalizadas podem ser chamadas em contextos e arquiteturas arbitrários.
* As camadas podem ter parâmetros locais, que podem ser criados por meio de funções integradas.


## Exercícios

3. Projete uma camada que recebe uma entrada e calcula uma redução de tensor,
    ou seja, ele retorna $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
4. Projete uma camada que retorne a metade anterior dos coeficientes de Fourier dos dados.


[Discussão](https://discuss.d2l.ai/t/58)


<!--stackedit_data:
eyJoaXN0b3J5IjpbMTE0MTM0NDQ0OSw2MzczMzk3NTZdfQ==
-->
